Analysis based on wards

In [1]:
from random import shuffle, randint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, LineString, mapping, shape
from descartes import PolygonPatch
import random
import fiona
import numpy as np
import csv
from fiona import collection

import geopandas as gpd
from geopandas.tools import sjoin # rtree index in-build, used with inner, intersection
import pandas as pd

from collections import defaultdict

In [9]:
def find_intersections(o):
    
    from collections import defaultdict

    paired_ind = [o.index_PP, o.index_WARD]

    d_over_ind = defaultdict(list)

    # creating a dictionary that has prescints as keys and associated small areas as values
    for i in range(len(paired_ind[0].values)):
        if not paired_ind[0].values[i]==paired_ind[1].values[i]: # it shows itself as intersection
            d_over_ind[paired_ind[0].values[i]].append(paired_ind[1].values[i])

    # get rid of the pol precincts with no small areas associated to them- not the most efficient way
    d_temp = {}
    for l in d_over_ind:
        if len(d_over_ind[l]):
            d_temp[l] = d_over_ind[l]

    return d_temp
      
def calculate_join_indices(g1_reind, g2_reind):
       
        # the indexing:
        out = sjoin(g1_reind, g2_reind, how ="inner", op = "intersects")
        
        out.drop('index_right', axis=1, inplace=True) # one index doubles, so we drop one
      
        # output retains only 1 area (left or right join), and gives no intersection area.
        # so we create an array with paired indices: police precincts with a ward
        # we use it in a loop in a function below
        dict_over_ind = find_intersections(out) 
        
        return dict_over_ind
    

In [ ]:
def calculate_join(dict_over_ind, g1_reind, g2_reind):
        area_total = 0
        data_aggreg = []

        # note to self: make sure to import shapely Polygon
        for index1, crim in g1_reind.iterrows():
            try:
                index1 = crim.pp_index
                sals_found = dict_over_ind[index1]

                for sal in range(len(sals_found)):
                    pom = g2_reind[g2_reind.sal_index == sals_found[sal]]['geometry']        

                    #if pom.intersects(crim['geometry']).values[0]:
                    area_int = pom.intersection(crim['geometry']).area.values[0]
                    if area_int>0:
                        area_total += area_int 
                        area_crim = crim['geometry'].area

                        area_popu = pom.values[0].area

                        popu_count = g2_reind[g2_reind.sal_index == sals_found[sal]]['PPL_CNT'].values[0]
                        murd_count = crim['murd_cnt']
                        pol_province = crim['province']
                        popu_frac = (area_int / area_popu) * popu_count# fraction of the pop area contained inside the crim
                        #print(popu_frac)
                        extra_info_col_names = ['DC_NAME','MN_NAME','MP_NAME','PR_NAME','SP_NAME']
                        
                        extra_info_col_codes = ['MN_CODE','MP_CODE','PR_CODE','SAL_CODE','SP_CODE']

                        extra_names = g2_reind[g2_reind.sal_index == sals_found[sal]][extra_info_col_names]#.filter(regex=("NAME"))
                        extra_codes = g2_reind[g2_reind.sal_index == sals_found[sal]][extra_info_col_codes]#.filter(regex=("NAME"))

                        data_aggreg.append({'geometry': pom.intersection(crim['geometry']).values[0], 'id1': index1,\
                                      'id2': sals_found[sal] ,'area_pp': area_crim,'area_sal': area_popu,\
                                  'area_inter': area_int, 'popu_inter' : popu_frac, 'popu_sal': popu_count,\
                                  'murd_cnt': murd_count,'province': pol_province,
                                  'DC_NAME': extra_names.DC_NAME.values[0],\
                                  'MN_NAME': extra_names.MN_NAME.values[0], 'MP_NAME': extra_names.MP_NAME.values[0],\
                                  'PR_NAME': extra_names.PR_NAME.values[0],'SP_NAME': extra_names.SP_NAME.values[0],\
                                  'MN_CODE': extra_codes.MN_CODE.values[0],'MP_CODE': extra_codes.MP_CODE.values[0],\
                                  'PR_CODE': extra_codes.PR_CODE.values[0],'SAL_CODE': extra_codes.SAL_CODE.values[0],\
                                  'SP_CODE': extra_codes.SP_CODE.values[0]} )
            except:
                pass
            
        df_t = gpd.GeoDataFrame(data_aggreg,columns=['geometry', 'id1','id2','area_pp',\
                                       'area_sal','area_inter', 'popu_inter',\
                                       'popu_sal', 'murd_cnt','province','DC_NAME',\
                                       'MN_NAME','MP_NAME','PR_NAME','SP_NAME',\
                                      'MN_CODE','MP_CODE','PR_CODE','SAL_CODE','SP_CODE'])
        #df_t.to_file(out_name)
        return df_t, area_total, data_aggreg

In [2]:
# this function adds the remaining columns, calculates fractions etc
def compute_final_col(df_temp):
    # add population data per police percinct to the main table
    # id1- PP, id2 - SAL
    temp = df_temp.groupby(by=['id1'])['popu_inter'].sum().reset_index()

    data_with_population = pd.merge(df_temp, temp, on='id1', how='outer')\
            .rename(columns={'popu_inter_y':'popu_frac_per_pp', 'popu_inter_x':'popu_inter'})

    # finally, update the murder rate per SAL : id2 is sal's id    

    data_with_population['murd_est_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\
               * data_with_population['murd_cnt']
    data_mur_per_int = data_with_population.groupby(by=['id2'])['murd_est_per_int'].sum().reset_index()

    data_mur_per_sal = data_mur_per_int.rename(columns={'murd_est_per_int':'murd_est_per_sal'})

    data_with_population['ratio_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\

    data_complete = pd.merge(data_with_population, data_mur_per_sal, on='id2', how='outer')\
            .rename(columns={'id1':'index_PP', 'id2':'index_SAL'})
    return data_complete

LOAD the data

In [11]:
#ppSHP = 'shapefiles/updated/polPrec_murd2015_prov_aea.shp'
#warSHP = '../maps/data/Wards2011_aea.shp'

#geo_ward = gpd.GeoDataFrame.from_file(warSHP)
#geo_pp = gpd.GeoDataFrame.from_file(ppSHP)

geo_pp_reind = geo_pp.reset_index().rename(columns={'index':'index_PP'})
geo_ward_reind = geo_ward.iloc[:,[2,3,7,8,9]].reset_index().rename(columns={'index':'index_WARD'})

out = sjoin(geo_pp_reind,geo_ward_reind, how ="inner", op = "intersects")
    
#dict_int = calculate_join_indices(geo_pol_reind,geo_sal_reind)

In [12]:
out

,index_WARD,MUNICNAME,PROVINCE,WARD_ID,WARD_POP,geometry,index_right,index_PP,compnt_nm,create_dt,murd_cnt,province,version
0,0,Matzikama Local Municipality,Western Cape,10101001,3197,POLYGON ((-685228.0889374854 -3589226.72492899...,743,743,vredendal,20140207,8,Western Cape,1.1.0
1,1,Matzikama Local Municipality,Western Cape,10101002,3256,POLYGON ((-691993.9985869445 -3593573.14527526...,743,743,vredendal,20140207,8,Western Cape,1.1.0
2,2,Matzikama Local Municipality,Western Cape,10101003,3268,POLYGON ((-658413.9011690317 -3596329.85491053...,743,743,vredendal,20140207,8,Western Cape,1.1.0
3,3,Matzikama Local Municipality,Western Cape,10101004,3110,POLYGON ((-655557.0895257025 -3602909.24873437...,743,743,vredendal,20140207,8,Western Cape,1.1.0
4,4,Matzikama Local Municipality,Western Cape,10101005,3122,POLYGON ((-661872.4750124722 -3603570.70495011...,743,743,vredendal,20140207,8,Western Cape,1.1.0
6,6,Matzikama Local Municipality,Western Cape,10101007,3663,POLYGON ((-621849.4420089257 -3589904.99535903...,743,743,vredendal,20140207,8,Western Cape,1.1.0
7,7,Matzikama Local Municipality,Western Cape,10101008,3354,POLYGON ((-635471.8895401303 -3493308.89642394...,743,743,vredendal,20140207,8,Western Cape,1.1.0
0,0,Matzikama Local Municipality,Western Cape,10101001,3197,POLYGON ((-685228.0889374854 -3589226.72492899...,767,767,lutzville,20140207,12,Western Cape,1.1.0
1,1,Matzikama Local Municipality,Western Cape,10101002,3256,POLYGON ((-691993.9985869445 -3593573.14527526...,767,767,lutzville,20140207,12,Western Cape,1.1.0
2,2,Matzikama Local Municipality,Western Cape,10101003,3268,POLYGON ((-658413.9011690317 -3596329.85491053...,767,767,lutzville,20140207,12,Western Cape,1.1.0


In [7]:
geo_ward.iloc[:,[2,3,7,8,9]].reset_index().head()

,index,MUNICNAME,PROVINCE,WARD_ID,WARD_POP,geometry
0,0,Matzikama Local Municipality,Western Cape,10101001,3197,POLYGON ((-685228.0889374854 -3589226.72492899...
1,1,Matzikama Local Municipality,Western Cape,10101002,3256,POLYGON ((-691993.9985869445 -3593573.14527526...
2,2,Matzikama Local Municipality,Western Cape,10101003,3268,POLYGON ((-658413.9011690317 -3596329.85491053...
3,3,Matzikama Local Municipality,Western Cape,10101004,3110,POLYGON ((-655557.0895257025 -3602909.24873437...
4,4,Matzikama Local Municipality,Western Cape,10101005,3122,POLYGON ((-661872.4750124722 -3603570.70495011...
